# Causal Language Modeling Task
A series of experiments demonstrating causal language modeling and training performance on a Reformer model. Model, datasets, and examples sourced from Huggingface.


---


## Test Models
**Reformer**
* 6-layer
* 256-hidden
* 2-heads
* 3M parameters
* Trained on English text: Crime and Punishment novel by Fyodor Dostoyevsky.


In [ ]:
!pip install datasets transformers sentencepiece

In [2]:
# Imports
import math, random, torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [4]:
def tokenize_function_ptb(examples):
    return tokenizer(examples["sentence"])

def tokenize_function_wt2_enwik8(examples):
    return tokenizer(examples["text"])
    
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

def prep_enwik8(path):
    file = path + 'enwik8'
    
    # Read file
    with open(file) as f:
        lines = f.readlines()
        
    random.shuffle(lines)

    # Calculate splits: 80/10/10 - train/val/test
    train_split = math.floor(len(lines)*.8)
    test_val_split = math.floor(len(lines)*.1)
    
    with open(path+'enwik8_train.txt', 'w') as train:
        with open(path+'enwik8_validation.txt', 'w') as val:
            with open(path+'enwik8_test.txt', 'w') as test:
                for i, line in enumerate(lines):
                    if i < train_split:
                        train.write(line)
                    elif i < train_split + test_val_split:
                        val.write(line)
                    else:
                        test.write(line)

In [5]:
# Hyperparameters
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 30
BATCH_SIZE = 16
block_size = 16384
PUSH_HUB = False
AXIAL_POS = False

# Dataset selection
DATASET_SELECT = 0  # 0 = wikitext-2, 1 = penn treebank, 2 = enwik8
PATH_TO_ENWIK8 = '/content/data/' 

In [6]:
model_id = 'google/reformer-crime-and-punishment'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, padding=True)

In [ ]:
if DATASET_SELECT == 0:
  datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
  tokenized_datasets = datasets.map(tokenize_function_wt2_enwik8, batched=True, num_proc=4, remove_columns=["text"])
elif DATASET_SELECT == 1:
  datasets = load_dataset("ptb_text_only")
  tokenized_datasets = datasets.map(tokenize_function_ptb, batched=True, num_proc=4, remove_columns=["sentence"])
elif DATASET_SELECT == 2:
  prep_enwik8(PATH_TO_ENWIK8)
  datasets = load_dataset('text', data_files={'train': PATH_TO_ENWIK8+'enwik8_train.txt','validation': PATH_TO_ENWIK8+'enwik8_validation.txt','test': PATH_TO_ENWIK8+'enwik8_test.txt'})
  tokenized_datasets = datasets.map(tokenize_function_wt2_enwik8, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
#tokenized_datasets["train"][1]

In [9]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
#tokenizer.decode(lm_datasets["train"][1]["input_ids"])

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, axial_pos_embds=AXIAL_POS).to(device)

In [11]:
model_name = model_id.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    adafactor=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=PUSH_HUB,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
train_results = trainer.train()

In [ ]:
#trainer.save_model()

In [ ]:
# Print Perplexity
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
# Display Metrics
metrics = train_results.metrics
trainer.log_metrics("train", metrics)

# Memory Benchmarks
## Test Models
**Reformer**
* 6-layer
* 256-hidden
* 2-heads
* 3M parameters
* Trained on English text: Crime and Punishment novel by Fyodor Dostoyevsky.  

**Reformer**
* 12-layer
* 1024-hidden
* 8-heads
* 149M parameters
* Trained on English Wikipedia data - enwik8.

**GPT2 Base**
* 12-layer
* 768-hidden
* 12-heads
* 117M parameters
* OpenAI GPT-2 English model

In [2]:
!pip -qq install transformers py3nvml

In [ ]:
from transformers import GPT2Config, ReformerConfig, PyTorchBenchmark, PyTorchBenchmarkArguments

In [23]:
config_cp_no_pos_axial_embeds = ReformerConfig.from_pretrained("google/reformer-crime-and-punishment", axial_pos_embds=False)  # disable axial positional embeddings
config_cp_pos_axial_embeds = ReformerConfig.from_pretrained("google/reformer-crime-and-punishment", axial_pos_embds=True, axial_pos_embds_dim=(64, 192), axial_pos_shape=(512, 1024))  # enable axial positional embeddings

benchmark_args = PyTorchBenchmarkArguments(sequence_lengths=[256, 512, 2048, 16384], batch_sizes=[16], models=["Reformer C/P"], speed=False, env_print=False)

In [ ]:
benchmark = PyTorchBenchmark(configs=[config_cp_no_pos_axial_embeds], args=benchmark_args)
result = benchmark.run()

In [ ]:
benchmark = PyTorchBenchmark(configs=[config_cp_pos_axial_embeds], args=benchmark_args)
result = benchmark.run()

In [ ]:
config_en8_no_pos_axial_embeds = ReformerConfig.from_pretrained("google/reformer-enwik8", axial_pos_embds=False)  # disable axial positional embeddings
benchmark_args = PyTorchBenchmarkArguments(sequence_lengths=[256, 512, 1024, 2048], batch_sizes=[16], models=["Reformer en8"], speed=False, env_print=False)

benchmark = PyTorchBenchmark(configs=[config_en8_no_pos_axial_embeds], args=benchmark_args)
result = benchmark.run()

In [ ]:
config_gpt2 = GPT2Config.from_pretrained("gpt2")
benchmark_args = PyTorchBenchmarkArguments(sequence_lengths=[256, 512, 1024, 2048], batch_sizes=[16], models=["GPT2 Base"], speed=False, env_print=False)

benchmark = PyTorchBenchmark(configs=[config_gpt2], args=benchmark_args)
result = benchmark.run()
